In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re, string
from konlpy.tag import Komoran
from collections import Counter

komoran = Komoran()

[0.004s][warning][os,thread] Attempt to protect stack guard pages failed (0x0000000168f90000-0x0000000168f9c000).
[0.004s][warning][os,thread] Attempt to deallocate stack guard pages failed.


## 크롤링 데이터

In [3]:
cafe = pd.read_csv('./kakao_review/kakao_cafe.csv', encoding='UTF-8')
motel = pd.read_csv('./kakao_review/kakao_motel.csv', encoding='UTF-8')
pension = pd.read_csv('./kakao_review/kakao_pension.csv', encoding='UTF-8')
resort = pd.read_csv('./kakao_review/kakao_resort.csv', encoding='UTF-8')
food = pd.read_csv('./kakao_review/kakao_food.csv', encoding='UTF-8')
tour = pd.read_csv('./kakao_review/kakao_tour.csv', encoding='UTF-8')
hotel = pd.read_csv('./kakao_review/kakao_hotel.csv', encoding='UTF-8')

In [66]:
print(f"카페의 수는 {cafe.shape[0]}개입니다.")
print(f"모텔의 수는 {motel.shape[0]}개입니다.")
print(f"펜션의 수는 {pension.shape[0]}개입니다.")
print(f"리조트의 수는 {resort.shape[0]}개입니다.")
print(f"식당의 수는 {food.shape[0]}개입니다.")
print(f"관광지의 수는 {tour.shape[0]}개입니다.")
print(f"호텔의 수는 {hotel.shape[0]}개입니다.")

카페의 수는 410개입니다.
모텔의 수는 95개입니다.
펜션의 수는 404개입니다.
리조트의 수는 74개입니다.
식당의 수는 304개입니다.
관광지의 수는 392개입니다.
호텔의 수는 170개입니다.


In [22]:
# 정규화
def preprocess(text):
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text


# 명사 추출, 한글자 제외, 불용어 제거
def final(text):
    n = []
    word = komoran.nouns(text)
    for w in word:
        if len(w)>1 and w not in stopwords:
            n.append(w)
    return " ".join(n)

# 최종
def finalpreprocess(text):
    return final(preprocess(text))

##  카페

In [14]:
cafe.head()

,카페명,분류,주소,별점,별점페이지,별점갯수,명사
0,원앤온리,카페,제주특별자치도 서귀포시 안덕면 산방로 141 원앤온리,4.0,https://place.map.kakao.com/217787831#comment,466,시설 보니 커피 만원 하네 그라도 카페 취향 느낌 인상 제주 처음 사람 풍경 마운틴...
1,퍼시픽리솜 더클리프카페&펍,카페,제주특별자치도 서귀포시 중문관광로 154-17,3.8,https://place.map.kakao.com/1378746048#comment,359,이국 분위기 음료 테이 피자 가격 일요일 영업 명시 영업시간 닭다리 살로 현무암 치...
2,카페나모나모,디저트카페,제주특별자치도 제주시 도두봉6길 4 1~4층,4.6,https://place.map.kakao.com/373077516#comment,351,음료 생각 날씨 창문 사방 개방 힐링 하기 무지개 해안 도로 주차공간 후기 카페 제...
3,울트라마린,커피전문점,제주특별자치도 제주시 한경면 일주서로 4611 지하1층,4.5,https://place.map.kakao.com/1915903142#comment,253,분위기 커피 제주도 여행 키즈 분위기 최고 음료 공간 커피 맛집 커피 산미 티라미수...
4,드르쿰다in성산,테마카페,제주특별자치도 서귀포시 성산읍 섭지코지로25번길 64,4.5,https://place.map.kakao.com/795878887#comment,239,메가 실망 사진 별점 알바 직원 친절 오바 리뷰 텐트 안도 후기 별점 주작 캐슬 회...


In [15]:
cafe

,카페명,분류,주소,별점,별점페이지,별점갯수,명사
0,원앤온리,카페,제주특별자치도 서귀포시 안덕면 산방로 141 원앤온리,4.0,https://place.map.kakao.com/217787831#comment,466,시설 보니 커피 만원 하네 그라도 카페 취향 느낌 인상 제주 처음 사람 풍경 마운틴...
1,퍼시픽리솜 더클리프카페&펍,카페,제주특별자치도 서귀포시 중문관광로 154-17,3.8,https://place.map.kakao.com/1378746048#comment,359,이국 분위기 음료 테이 피자 가격 일요일 영업 명시 영업시간 닭다리 살로 현무암 치...
2,카페나모나모,디저트카페,제주특별자치도 제주시 도두봉6길 4 1~4층,4.6,https://place.map.kakao.com/373077516#comment,351,음료 생각 날씨 창문 사방 개방 힐링 하기 무지개 해안 도로 주차공간 후기 카페 제...
3,울트라마린,커피전문점,제주특별자치도 제주시 한경면 일주서로 4611 지하1층,4.5,https://place.map.kakao.com/1915903142#comment,253,분위기 커피 제주도 여행 키즈 분위기 최고 음료 공간 커피 맛집 커피 산미 티라미수...
4,드르쿰다in성산,테마카페,제주특별자치도 서귀포시 성산읍 섭지코지로25번길 64,4.5,https://place.map.kakao.com/795878887#comment,239,메가 실망 사진 별점 알바 직원 친절 오바 리뷰 텐트 안도 후기 별점 주작 캐슬 회...
...,...,...,...,...,...,...,...
405,프랑제리 켄싱턴리조트 중문점,"제과,베이커리",제주특별자치도 서귀포시 중문관광로72번길 29-29 3층,5.0,https://place.map.kakao.com/1671698321#comment,2,사과 당근 샌드위치 대박 사이즈 가격
406,더벤티 해비치리조트점,커피전문점,제주특별자치도 서귀포시 표선면 민속해안로 577 1층,5.0,https://place.map.kakao.com/1768944605#comment,4,친절 덕분 충전 사장 친절 음료 저가 프렌 이즈 빨대 사장 필요 사람 시스템 친절 가격
407,호텔샌드,카페,제주특별자치도 제주시 한림읍 한림로 339,3.9,https://place.map.kakao.com/1500112810#comment,56,저녁 바다 풍경 해수욕장 한눈 맛집 음료 디저트 인테리어 예뻐서 용서 이국 분위기 ...
408,해비치호텔앤드리조트 마고,"제과,베이커리",제주특별자치도 서귀포시 표선면 민속해안로 537 1층,4.4,https://place.map.kakao.com/21573685#comment,17,방문 이것저것 클레어 쇼콜라 카야 크림 임직원 예약 조식 눈물 해변 한잔 클레어 천...


In [16]:
with open('./stopwords.txt', 'r', encoding='cp949') as f:
    list_file = f.readlines() 
stopwords = list_file[0].split(",")

In [8]:
cafe['명사'] = cafe['리뷰'].apply(lambda x: finalpreprocess(x))

In [9]:
cafe.drop('리뷰', axis=1, inplace=True)

In [10]:
cafe_keyword = pd.DataFrame()
cafe_keyword['place'] = cafe['카페명']

### 가장 많이 등장하는 5개의 키워드 뽑기

In [11]:
cafe_keyword['Top5'] = np.nan

for i in range(len(cafe)):
    noun = cafe['명사'][i].split(" ")
    count = Counter(noun).most_common(5)
    top5 = []
    for j in range(len(count)):
        top5.append(count[j][0])
    cafe_keyword['Top5'][i] = top5

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_2462/3303302687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_keyword['Top5'][i] = top5


In [12]:
cafe_keyword

,place,Top5
0,원앤온리,"[카페, 가격, 커피, 산방산, 음료]"
1,퍼시픽리솜 더클리프카페&펍,"[분위기, 가격, 음식, 음료, 칵테일]"
2,카페나모나모,"[커피, 카페, 디저트, 분위기, 음료]"
3,울트라마린,"[친절, 커피, 카페, 사장, 직원]"
4,드르쿰다in성산,"[사진, 음료, 카페, 포토, 해변]"
...,...,...
405,프랑제리 켄싱턴리조트 중문점,"[사과, 당근, 샌드위치, 대박, 사이즈]"
406,더벤티 해비치리조트점,"[친절, 사장, 덕분, 충전, 음료]"
407,호텔샌드,"[음료, 분위기, 가격, 디저트, 인테리어]"
408,해비치호텔앤드리조트 마고,"[가격, 카야, 크림, 브레드, 추천]"


### 키워드 등장 횟수 뽑기

In [17]:
def count_keyword(text, keyword):
    noun = text.split(" ")
    num = Counter(noun)[keyword]
    return num

In [18]:
# for key in ['무료', '혜택', '친환경', '추천', '식재료', '제주산', '에너지', '환경', '비건', '야경']:
for key in ['야경']:
    cafe_keyword[key] = cafe['명사'].apply(lambda x: count_keyword(x, key))

In [20]:
cafe_keyword.sort_values('야경', ascending=False)

,place,Top5,야경
2,카페나모나모,"[커피, 카페, 디저트, 분위기, 음료]",4
4,드르쿰다in성산,"[사진, 음료, 카페, 포토, 해변]",4
123,73st,"[음료, 카페, 직원, 친절, 음식]",2
309,니나수족욕카페,"[카페, 족욕, 피로, 사장, 친절]",1
1,퍼시픽리솜 더클리프카페&펍,"[분위기, 가격, 음식, 음료, 칵테일]",1
...,...,...,...
135,노바운더리 제주,"[커피, 방문, 프라스틱, 카페, 플라스틱]",0
134,알마커피제작소,"[커피, 친절, 사장, 카페, 분위기]",0
133,동명정류장,"[사장, 친절, 카페, 사진, 음료]",0
132,누옥,"[커피, 카페, 타르트, 분위기, 에그]",0


In [21]:
cafe_keyword.to_csv('./etc_data/cafe_eco.csv', header=True, index=False, encoding="UTF-8")

## 모텔

In [24]:
motel['명사'] = np.nan

for i in range(len(motel)):
    t = motel['리뷰'][i]
    try:
        motel['명사'][i] = finalpreprocess(t)
    except:
        motel['명사'][i] = " "

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/1227544122.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  motel['명사'][i] = finalpreprocess(t)


In [25]:
motel_keyword = pd.DataFrame()

motel_keyword['place'] = motel['카페명']
motel_keyword['Top5'] = np.nan

for i in range(len(motel)):
    noun = motel['명사'][i].split(" ")
    count = Counter(noun).most_common(5)
    top5 = []
    for j in range(len(count)):
        top5.append(count[j][0])
    motel_keyword['Top5'][i] = top5

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/3115805272.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  motel_keyword['Top5'][i] = top5


In [26]:
for key in ['무료', '혜택', '친환경', '추천', '식재료', '제주산', '에너지', '환경', '비건']:
    motel_keyword[key] = motel['명사'].apply(lambda x: count_keyword(x, key))

In [27]:
motel_keyword

,place,Top5,무료,혜택,친환경,추천,식재료,제주산,에너지,환경,비건
0,마레보비치호텔,"[조식, 호텔, 샤워, 시설, 친절]",0,0,0,2,0,0,0,0,0
1,제주엘루이호텔,"[화장실, 침대, 곰팡이, 냄새, 현관]",0,0,0,0,0,0,0,0,0
2,신신호텔 서귀포점,"[호텔, 직원, 주차장, 숙소, 친절]",0,0,0,2,0,0,0,0,0
3,코자호텔,"[냄새, 스파, 담배, 만족, 최고]",0,0,0,0,0,0,0,0,0
4,아인스호스텔,"[성비, 친절, 가격, 호텔, 이용]",0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
90,수모텔,"[사진, 인증, 외관, 사장, 친절]",0,0,0,0,0,0,0,0,0
91,폭포장모텔,[],0,0,0,0,0,0,0,0,0
92,만복모텔,"[이불, 흰색]",0,0,0,0,0,0,0,0,0
93,대정모텔,"[가족, 사장, 만족, 편의점, 가까이]",0,0,0,0,0,0,0,0,0


## 펜션

In [28]:
pension['명사'] = np.nan

for i in range(len(pension)):
    t = pension['리뷰'][i]
    try:
        pension['명사'][i] = finalpreprocess(t)
    except:
        pension['명사'][i] = " "

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/3201699190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pension['명사'][i] = finalpreprocess(t)


In [29]:
pension

,카페명,분류,주소,별점,별점페이지,별점갯수,리뷰,명사
0,미르빌펜션&리조트,펜션,제주특별자치도 서귀포시 안덕면 사계로114번길 87,3.6,https://place.map.kakao.com/11320373#comment,16,"[[1, '그건 고객님 사정이죠 라고 너무 차갑게 전화받는 태도가 공감 능력 결여된...",고객 사정 전화 태도 공감 능력 결여 서비스 마인드 제로 도착 기분 자전거 여행 자...
1,에코그린리조트,펜션,제주특별자치도 제주시 한림읍 귀덕6길 94,4.1,https://place.map.kakao.com/24156460#comment,22,"[[4, '조용해서 좋아요. 그러나 발코니 창은 자주 청소해 줬으면 좋겠어요. 위치...",발코니 청소 위치 시골 방충망 유리 벌레 인해 창문 기분 싸구려 여인숙 느낌 루프 ...
2,라임오렌지빌,펜션,제주특별자치도 서귀포시 칠십리로 332,4.0,https://place.map.kakao.com/8341593#comment,4,"[[5, '커피가 너무~~~~~ 맛있어요 ^^'], [5, ''], [2, '조상복...",커피 조상 복합 가족 층집 구조 시설 화장실 당시 직원 친절 아침 창문 너머 풍경
3,풀빌라소랑,펜션,제주특별자치도 제주시 조천읍 곱은달남길 218,4.0,https://place.map.kakao.com/1461253252#comment,15,"[[4, '조식이 생각보다 괜찮았고, 독채라서 조용했어요.\n외진 데라 위치가 아쉽...",조식 생각 위치 벌레 기억 기록 직원 친절 조식 정수기 편리 보니 풀장 수질 다음번...
4,통나무파크,펜션,제주특별자치도 제주시 애월읍 도치돌길 293,4.4,https://place.map.kakao.com/21355370#comment,14,"[[1, '이불은 누렇고 숙소 입구는 거미줄 쳐있고 폐자재 널려있고 왜 싼가 했더니...",이불 숙소 입구 거미줄 자재 이유 숙소 컨디션 실전 에어컨 방도 업그레이드 친구 커...
...,...,...,...,...,...,...,...,...
399,뷰티플리조트,펜션,제주특별자치도 서귀포시 이어도로253번길 2,5.0,https://place.map.kakao.com/1983547721#comment,1,"[[5, '넘 좋아요~']]",
400,길리리조트,펜션,제주특별자치도 제주시 한림읍 명재로 100-47,5.0,https://place.map.kakao.com/1889502414#comment,1,"[[5, '오픈한지 얼마 되지 않아 객실 컨디션이 너무나도 훌륭함!\n전 객실에 네...",오픈 한지 객실 컨디션 객실 네스프레소 머신 코웨이 정수기 센세이션 전기자동차 충전...
401,대웅황토리조트,펜션,제주특별자치도 제주시 애월읍 번드르길 27,5.0,https://place.map.kakao.com/27200348#comment,1,"[[5, '']]",
402,더시크릿서귀포리조트&빌라,펜션,제주특별자치도 서귀포시 대정읍 노을해안로 216-6,5.0,https://place.map.kakao.com/1880193980#comment,3,"[[5, '숙소를 늦게 예약해서 어딜 해야하나 걱정했는데 마침 풀빌라치고 저렴한 가...",숙소 예약 걱정 빌라 가격 숙소 친절 사장 개별 수영장 사이즈 커서 방문 의사 개장...


In [30]:
pension_keyword = pd.DataFrame()

pension_keyword['place'] = pension['카페명']
pension_keyword['Top5'] = np.nan

for i in range(len(pension)):
    noun = pension['명사'][i].split(" ")
    count = Counter(noun).most_common(5)
    top5 = []
    for j in range(len(count)):
        top5.append(count[j][0])
    pension_keyword['Top5'][i] = top5
    
for key in ['무료', '혜택', '친환경', '추천', '식재료', '제주산', '에너지', '환경', '비건']:
    pension_keyword[key] = pension['명사'].apply(lambda x: count_keyword(x, key))

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/4002347720.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pension_keyword['Top5'][i] = top5


In [31]:
pension_keyword

,place,Top5,무료,혜택,친환경,추천,식재료,제주산,에너지,환경,비건
0,미르빌펜션&리조트,"[산방산, 제주, 가격, 성비, 전화]",0,0,0,0,0,0,0,0,0
1,에코그린리조트,"[수영장, 직원, 루프, 청소, 시트]",0,0,0,1,0,0,0,1,0
2,라임오렌지빌,"[커피, 조상, 복합, 가족, 층집]",0,0,0,0,0,0,0,0,0
3,풀빌라소랑,"[청소, 가격, 조식, 직원, 상태]",1,0,0,0,0,0,0,0,0
4,통나무파크,"[숙소, 알파카, 컨디션, 침대, 목장]",0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
399,뷰티플리조트,[],0,0,0,0,0,0,0,0,0
400,길리리조트,"[객실, 오픈, 한지, 컨디션, 네스프레소]",0,0,0,0,0,0,0,0,0
401,대웅황토리조트,[],0,0,0,0,0,0,0,0,0
402,더시크릿서귀포리조트&빌라,"[숙소, 예약, 걱정, 빌라, 가격]",0,0,0,0,0,0,0,0,0


## 리조트

In [32]:
resort['명사'] = np.nan

for i in range(len(resort)):
    t = resort['리뷰'][i]
    try:
        resort['명사'][i] = finalpreprocess(t)
    except:
        resort['명사'][i] = " "

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/2640672236.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resort['명사'][i] = finalpreprocess(t)


In [33]:
resort

,카페명,분류,주소,별점,별점페이지,별점갯수,리뷰,명사
0,휘닉스제주,"콘도,리조트",제주특별자치도 서귀포시 성산읍 섭지코지로 107,3.8,https://place.map.kakao.com/8177478#comment,54,"[[5, '바다뷰는 아니었지만 바로 앞에 섭지코지로 산책 나가기가 너무 좋아요\n새...",섭지코지 산책 새벽바람 기분 조식 연식 시설 노후 대가족 만족 수영장 입장 제한 횟...
1,한화리조트 제주,"콘도,리조트",제주특별자치도 제주시 명림로 575-107,3.8,https://place.map.kakao.com/8016361#comment,36,"[[1, '오래된건 이해 합니다.\n알고 왔으니깐요.\n그런데 청결은 정말 최악입니...",이해 청결 최악 욕실 여기저기 머리카락 곰팡이 온돌 이불 이불 먼지 가격 리조트 느...
2,소노벨제주 신관,"콘도,리조트",제주특별자치도 제주시 조천읍 신북로 577,3.5,https://place.map.kakao.com/27446161#comment,2,"[[2, '침구류 등이 불편\n'], [5, '신관']]",침구 불편 신관
3,켄싱턴리조트 제주중문,"콘도,리조트",제주특별자치도 서귀포시 중문관광로72번길 29-29,4.0,https://place.map.kakao.com/796916033#comment,9,"[[3, ''], [4, '여행 둘째날에 켄싱턴 한림에서 중문으로 넘어왔어요. \n...",여행 한림 중문 서비스 시설 한림 서비스 연휴 기존 직원 알바 생각 남자 직원 분이...
4,더큐브리조트 제주,"콘도,리조트",제주특별자치도 서귀포시 솔오름로105번길 24,3.8,https://place.map.kakao.com/230486137#comment,47,"[[4, '가성비를 생각하면 강추!\n청결은 별 5개중 3.5정도\n조식도 3....",성비 생각 청결 개중 조식 아주 추천 리조트 성비 최고 리조트 아이들 조식 방문 가...
...,...,...,...,...,...,...,...,...
69,제주캐슬마운틴리조트,"콘도,리조트",제주특별자치도 서귀포시 안덕면 산록남로 513-14,5.0,https://place.map.kakao.com/1748908555#comment,1,"[[5, '']]",
70,인피니티리조트,"콘도,리조트",제주특별자치도 제주시 조천읍 조함해안로 376-9,5.0,https://place.map.kakao.com/1915158888#comment,1,"[[5, '직원분들 다 매우 친절하시고 뷰도 좋음. 무엇보다 침구가 대박임 굉장히 ...",직원 친절 침구 대박 참고 투숙객 할인 스쿠버 다이빙 추천 성비 최고
71,벙커호텔앤리조트,"콘도,리조트",제주특별자치도 서귀포시 중산간동로8183번길 16,5.0,https://place.map.kakao.com/1551672902#comment,1,"[[5, '우도숙소를 예약했으나 결항되어 갑자기 당일 급 검색해서 들어온 호텔\n그...",우도 숙소 예약 당일 검색 호텔 기대 가격 박만 기대 침구 관리 도착 건물 정면 모...
72,벨라비타,"콘도,리조트",제주특별자치도 서귀포시 일주서로 809,5.0,https://place.map.kakao.com/1935009599#comment,1,"[[5, '']]",


In [34]:
resort_keyword = pd.DataFrame()

resort_keyword['place'] = resort['카페명']
resort_keyword['Top5'] = np.nan

for i in range(len(resort)):
    noun = resort['명사'][i].split(" ")
    count = Counter(noun).most_common(5)
    top5 = []
    for j in range(len(count)):
        top5.append(count[j][0])
    resort_keyword['Top5'][i] = top5
    
for key in ['무료', '혜택', '친환경', '추천', '식재료', '제주산', '에너지', '환경', '비건']:
    resort_keyword[key] = resort['명사'].apply(lambda x: count_keyword(x, key))

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/339075700.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resort_keyword['Top5'][i] = top5


In [35]:
resort_keyword

,place,Top5,무료,혜택,친환경,추천,식재료,제주산,에너지,환경,비건
0,휘닉스제주,"[조식, 직원, 섭지코지, 숙소, 전화]",0,0,0,1,0,0,0,1,0
1,한화리조트 제주,"[리조트, 조식, 시설, 느낌, 화장실]",0,0,0,2,0,0,0,1,0
2,소노벨제주 신관,"[침구, 불편, 신관]",0,0,0,0,0,0,0,0,0
3,켄싱턴리조트 제주중문,"[객실, 조식, 로비, 욕조, 한림]",0,0,0,1,0,0,0,2,0
4,더큐브리조트 제주,"[성비, 가격, 숙소, 객실, 시설]",0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
69,제주캐슬마운틴리조트,[],0,0,0,0,0,0,0,0,0
70,인피니티리조트,"[직원, 친절, 침구, 대박, 참고]",0,0,0,1,0,0,0,0,0
71,벙커호텔앤리조트,"[기대, 만족, 우도, 숙소, 예약]",0,0,0,0,0,0,0,0,0
72,벨라비타,[],0,0,0,0,0,0,0,0,0


## 식당

In [23]:
food['명사'] = np.nan

for i in range(len(food)):
    t = food['리뷰'][i]
    try:
        food['명사'][i] = finalpreprocess(t)
    except:
        food['명사'][i] = " "

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_2462/3979008968.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food['명사'][i] = finalpreprocess(t)


In [ ]:
food

,카페명,분류,주소,별점,별점페이지,별점갯수,리뷰,명사
0,연돈,"돈까스,우동",제주특별자치도 서귀포시 일주서로 968-10 1층,3.9,https://place.map.kakao.com/1890778114#comment,458,"[[3, '22.09.29(목) 방문.\n성산에서 두시간 걸려서 일정변경해가면서 갔...",방문 성산 일정 변경 성산 시내버스 정거장 레게 기억 결론 페이스 리뷰 유명한 나중...
1,숙성도 노형점,"육류,고기",제주특별자치도 제주시 원노형로 41,4.1,https://place.map.kakao.com/316010726#comment,423,"[[2, '너무 기대를했나봐요\n여기가려고 일부러 제주시내 1박 잡았는데\n시간이 ...",기대 제주시 목살 등심 숙성 숙성 목사 짜증 고기 목살 실망 숙성 고기 비계 고기 ...
2,우진해장국,해장국,제주특별자치도 제주시 서사로 11,3.7,https://place.map.kakao.com/11547525#comment,863,"[[3, '8:50에 줄서서 10:40분에 입장했습니다.\n\n고사리육개장을 먹었는...",서서 입장 고사리 육개장 묘사 추어탕 생선 소고기 고사리 된장 장과 청국장 걱정 복...
3,고집돌우럭 제주공항점,"해물,생선",제주특별자치도 제주시 임항로 30,4.8,https://place.map.kakao.com/232056222#comment,986,"[[5, '너무너무너무 맛있어요.. 요번 여행에서 여기가 제일 맛있었고 다음에 또 ...",요번 여행 우럭 양념 환상 전복 대박 무도 최고 옥돔 서버 친절 대접 기분 남편 국...
4,고집돌우럭 함덕점,한식,제주특별자치도 제주시 조천읍 신북로 491-9 2층,4.9,https://place.map.kakao.com/28082185#comment,1090,"[[5, '바닷가 바로 앞이라서 바다 보면서 먹을 수 있어서 너무 좋아요! 해 쨍쨍...",바닷가 바다 친절 말씀 음식 우럭 조림 기와 무우 듯이 제주 맛집 식사 하기 우럭 ...
...,...,...,...,...,...,...,...,...
299,캠퍼트리호텔앤리조트 더뷰,양식,제주특별자치도 제주시 해안마을서4길 100,5.0,https://place.map.kakao.com/317582138#comment,2,"[[5, '아담하고 바다뷰 그리고 조용하고 감성적인 분위기로 그야말로 휴식을 하고 ...",감성 분위기 휴식 충전 호텔 리조트 숙소 커피 테이크아웃 최고 레스토랑 대박 유리창...
300,황가네뚝배기,한식,제주특별자치도 제주시 삼무로11길 24,3.9,https://place.map.kakao.com/14525030#comment,8,"[[5, '내.돈.내.산\n성게미역국 진짜 핵존맛이에요 전복뚝배기도JMT❤ 찐강추입...",미역국 전복 뚝배기 뚝배기 해물 부족 냉동 인지 조개 국물 리뷰 느낌 느낌 고등어 ...
301,씨에스호텔 씨에스다이닝,한식,제주특별자치도 서귀포시 중문관광로 198,5.0,https://place.map.kakao.com/351143149#comment,1,"[[5, '']]",
302,포도호텔 레스토랑,양식,제주특별자치도 서귀포시 안덕면 산록남로 863 1층,3.8,https://place.map.kakao.com/21508783#comment,24,"[[4, '새우튀김 우동 맛있지만 가격 생각하면 당연한 정도. 객주리조림은 쏘쏘. ...",새우 튀김 우동 가격 생각 객주 조림 식전 전복죽 스테이크 짬뽕 우동 별미 값어치 ...


In [ ]:
food_keyword = pd.DataFrame()

food_keyword['place'] = food['카페명']
food_keyword['Top5'] = np.nan

for i in range(len(food)):
    noun = food['명사'][i].split(" ")
    count = Counter(noun).most_common(5)
    top5 = []
    for j in range(len(count)):
        top5.append(count[j][0])
    food_keyword['Top5'][i] = top5
    
# for key in ['무료', '혜택', '친환경', '추천', '식재료', '제주산', '에너지', '환경', '비건', '아침']:
#     food_keyword[key] = food['명사'].apply(lambda x: count_keyword(x, key))

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/459106907.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_keyword['Top5'][i] = top5


In [ ]:
food_keyword

,place,Top5,무료,혜택,친환경,추천,식재료,제주산,에너지,환경,비건,아침
0,연돈,"[예약, 카레, 사람, 사장, 제주도]",0,0,0,10,0,0,0,0,0,8
1,숙성도 노형점,"[고기, 친절, 직원, 대기, 목살]",0,0,0,33,0,0,1,0,0,0
2,우진해장국,"[고사리, 해장국, 육개장, 대기, 친절]",4,0,0,30,0,0,0,1,0,52
3,고집돌우럭 제주공항점,"[우럭, 조림, 음식, 제주도, 제주]",4,0,0,79,0,0,0,0,0,11
4,고집돌우럭 함덕점,"[우럭, 음식, 조림, 방문, 함덕]",2,0,0,102,0,0,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
299,캠퍼트리호텔앤리조트 더뷰,"[커피, 감성, 분위기, 휴식, 충전]",0,0,0,0,0,0,0,0,0,0
300,황가네뚝배기,"[미역국, 뚝배기, 고등어, 전복, 구이]",0,0,0,1,0,0,0,0,0,2
301,씨에스호텔 씨에스다이닝,[],0,0,0,0,0,0,0,0,0,0
302,포도호텔 레스토랑,"[우동, 가격, 새우, 튀김, 호텔]",0,0,0,0,0,0,0,0,0,1


In [90]:
food_keyword[food_keyword['아침']>10]

,place,Top5,무료,혜택,친환경,추천,식재료,제주산,에너지,환경,비건,아침
2,우진해장국,"[고사리, 해장국, 육개장, 대기, 친절]",4,0,0,30,0,0,0,1,0,52
3,고집돌우럭 제주공항점,"[우럭, 조림, 음식, 제주도, 제주]",4,0,0,79,0,0,0,0,0,11
6,고집돌우럭 중문점,"[우럭, 조림, 음식, 방문, 제주도]",9,0,2,348,0,0,0,5,0,38
9,중문수두리보말칼국수,"[칼국수, 친절, 국물, 전복죽, 사장]",4,0,0,12,0,0,0,0,0,27
10,맛나식당,"[갈치, 조림, 고등어, 예약, 친절]",1,0,0,6,0,0,0,0,0,22
21,김희선제주몸국,"[고등어, 육개장, 고사리, 구이, 친절]",0,0,0,6,0,0,0,0,0,12
48,해월정,"[칼국수, 말죽, 친절, 음식, 가격]",0,0,0,33,0,0,1,0,0,15
126,용두암해촌,"[갈치, 조림, 구이, 전복, 고등어]",0,0,0,23,0,0,0,0,0,19
136,가람돌솥밥,"[솥밥, 전복, 재료, 갈치, 제주도]",1,0,0,13,0,0,0,0,0,12
137,성산일출봉 청운식당,"[갈치, 조림, 성산, 일출봉, 전복]",0,0,0,62,0,0,0,2,0,37


## 관광지

In [24]:
tour['명사'] = np.nan

for i in range(len(tour)):
    t = tour['리뷰'][i]
    try:
        tour['명사'][i] = finalpreprocess(t)
    except:
        tour['명사'][i] = " "

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_2462/1903289724.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tour['명사'][i] = finalpreprocess(t)


In [25]:
tour

,카페명,분류,주소,별점,별점페이지,별점갯수,리뷰,명사
0,올레길 7코스(서귀포-월평 올레),NaN,NaN,4.4,https://place.map.kakao.com/11635431#comment,32,"[[5, ''], [3, 'Soso'], [5, ''], [4, '초반에 나오는 천...",초반 천지연폭포 조망 리시 공원 우지 선녀탕 외돌개 이후 바닷길 풍경 감상 지루함 ...
1,올레길 1코스(시흥-광치기 올레),NaN,NaN,4.0,https://place.map.kakao.com/10626234#comment,15,"[[3, '정코스 추천. 역으로 가면 막판에 힘빠지는 코스가 등장한다.'], [5,...",코스 추천 막판 코스 등장 소요 처음 경치 달리 마을 성산 일출봉 처음 절경 사랑 ...
2,올레길 6코스(쇠소깍-서귀포 올레),NaN,NaN,4.9,https://place.map.kakao.com/8015963#comment,14,"[[5, ''], [5, '걸으면서 주변 구경하는 재미가 있었습니다. 풍경이 너무 ...",주변 구경 재미 풍경 쇠소깍 서귀포시 올레 여행자 센터 걷기 코스 출발 목포 자리 ...
3,올레길 18코스(제주원도심-조천 올레),NaN,NaN,4.7,https://place.map.kakao.com/13597626#comment,9,"[[5, '집에 가는날이라 잠깐 걸었지만 경치가 참 좋은 올레길~~~~! 담번엔 꼭...",경치 올레길 오랫동안 늦봄 집사람 올레 올레길 사라봉 별도 봉기 시비 코지 길이 기...
4,올레길 5코스(남원-쇠소깍 올레),NaN,NaN,4.7,https://place.map.kakao.com/11828668#comment,12,"[[3, '역으로걷늗데 풍경은좋습니다.\n근데 공중화장실 다 잠굴거면 화장실표시 왜...",풍경 공중화장실 화장실 표시 마을 경로당 화장실 기분 마을 난이도 여태 올레길 코스...
...,...,...,...,...,...,...,...,...
387,개다리폭포,폭포,제주특별자치도 서귀포시 색달동 3101-1,4.0,https://place.map.kakao.com/10129576#comment,4,"[[4, ''], [3, ''], [4, '가다가 넘어진 곳'], [5, '']]",
388,세리월드 카트레이싱,테마파크시설,제주특별자치도 서귀포시 법환동 881-2,3.5,https://place.map.kakao.com/11126147#comment,6,"[[5, '인터넷가 12500원임\n\n그리고 그 다음부터 계속타면 5천원 현장결제...",인터넷 천원 현장 결제 교육 포함 천원 시속 카트 차선 게임 드리프트 속도 직원 이...
389,톨칸이해변,"해수욕장,해변",제주특별자치도 제주시 우도면 연평리,4.0,https://place.map.kakao.com/20056498#comment,2,"[[5, '뷰가 예뻐요'], [3, '']]",
390,다랑쉬굴,유적지,제주특별자치도 제주시 구좌읍 세화리 2608-3,4.9,https://place.map.kakao.com/852336814#comment,8,"[[5, '접근성 제로, 일부러 왔는데\n차량 교행이 불가능한 농로에 대나무가 꺾...",접근성 제로 차량 농로 대나무 차량 진로 방해 다랑쉬오름 주차 도보 이동 비극 장소...


In [26]:
tour_keyword = pd.DataFrame()

tour_keyword['place'] = tour['카페명']
tour_keyword['Top5'] = np.nan

for i in range(len(tour)):
    noun = tour['명사'][i].split(" ")
    count = Counter(noun).most_common(5)
    top5 = []
    for j in range(len(count)):
        top5.append(count[j][0])
    tour_keyword['Top5'][i] = top5
    
for key in ['환경', '친환경', '야경']:
    tour_keyword[key] = tour['명사'].apply(lambda x: count_keyword(x, key))

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_2462/929351310.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tour_keyword['Top5'][i] = top5


In [27]:
tour_keyword

,place,Top5,환경,친환경,야경
0,올레길 7코스(서귀포-월평 올레),"[올레길, 코스, 풍경, 제주, 최고]",0,0,0
1,올레길 1코스(시흥-광치기 올레),"[코스, 오름, 달리, 처음, 성산]",0,0,0
2,올레길 6코스(쇠소깍-서귀포 올레),"[코스, 올레, 올레길, 쇠소깍, 거리]",0,0,0
3,올레길 18코스(제주원도심-조천 올레),"[코스, 올레길, 올레, 바다, 사라봉]",0,0,0
4,올레길 5코스(남원-쇠소깍 올레),"[코스, 산책, 올레길, 사진, 생각]",0,0,0
...,...,...,...,...,...
387,개다리폭포,[],0,0,0
388,세리월드 카트레이싱,"[카트, 천원, 핸들, 인터넷, 현장]",0,0,0
389,톨칸이해변,[],0,0,0
390,다랑쉬굴,"[차량, 다랑쉬오름, 역사, 사건, 희생자]",0,0,0


## 호텔

In [57]:
hotel['명사'] = np.nan

for i in range(len(hotel)):
    t = hotel['리뷰'][i]
    try:
        hotel['명사'][i] = finalpreprocess(t)
    except:
        hotel['명사'][i] = " "

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/3396550421.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel['명사'][i] = finalpreprocess(t)


In [58]:
hotel

,카페명,분류,주소,별점,별점페이지,별점갯수,리뷰,명사
0,라마다프라자 제주호텔,특급호텔,제주특별자치도 제주시 탑동로 66,3.7,https://place.map.kakao.com/15292731#comment,72,"[[5, '친절하고 깨끗합니다. 특히 지갑분실했는데 바로 연락오더군요. 너무너무 고...",친절 지갑 분실 연락 여행 미국 평점 구글 리뷰 예약 리뷰 리뷰 확인 후회 일다 호...
1,롯데호텔 제주,특급호텔,제주특별자치도 서귀포시 중문관광로72번길 35,4.3,https://place.map.kakao.com/7862728#comment,98,"[[5, '온수풀에서 아가들이랑 시간 보내기 너무 좋고 직원분들도 다들 친절하셔서 ...",온수 아가 직원 친절 기분 외부 조명 뷔페 만족 서비스 음식 메뉴 구성 제한 직원 ...
2,베니키아호텔 제주,호텔,제주특별자치도 제주시 애월읍 애월해안로 554-6,3.6,https://place.map.kakao.com/26517493#comment,77,"[[3, '매너, 서비스 부족한 직원. 경고 협박에 가까운 안내. \n실내 슬리퍼는...",매너 서비스 부족 직원 경고 협박 안내 실내 슬리퍼 동일 사이즈 사람 불편 입장 동...
3,베니키아중문호텔,호텔,제주특별자치도 서귀포시 천제연로 166,4.0,https://place.map.kakao.com/26854361#comment,11,"[[3, '아.. 주차가 정말&hellip; 지하는 7대 정도 밖에 안됩니다. 안타...",주차 지하 객실 제주도 성비 주차장 출장 성비 주차 주차장 만족 가격 교통 편리 관...
4,호텔더본 제주,호텔,제주특별자치도 서귀포시 색달로 18,4.1,https://place.map.kakao.com/1340230571#comment,88,"[[5, '가성비로 따지자면.. 최상급 인것 같아요..\n그렇게 넓지 않은 것이 단...",성비 상급 단점 일수 저녁 식사 아침 식사 합리 가격 해결 아이들 화장실 벽면 곰팡...
...,...,...,...,...,...,...,...,...
165,은하호텔,호텔,제주특별자치도 서귀포시 중앙로47번길 40,5.0,https://place.map.kakao.com/16672118#comment,1,"[[5, '2018 12/14 남자 3명 온돌방 2박했습니다\n\n한라산 등반으로 ...",남자 온돌방 한라산 등반 상황 가성 우수 은하 호텔 선택 가격 대비 온돌방 엘리베이...
166,라이프리버호텔,호텔,제주특별자치도 제주시 현사길 45 1~3층,5.0,https://place.map.kakao.com/2062435690#comment,1,NaN,
167,산방산호텔,숙박,제주특별자치도 서귀포시 대정읍 일주서로 2093,4.0,https://place.map.kakao.com/1977009698#comment,14,"[[5, '가성비 굿 '], [1, '벌레 나옵니다 많. 이 \n\n리뷰보고 기대 ...",성비 벌레 리뷰 기대 하구 직원 친절 호텔 청결 욕실 누가 사용 흔적 샤워 변기 머...
168,호텔케니모슬포,숙박,제주특별자치도 서귀포시 대정읍 하모중앙로 22,3.7,https://place.map.kakao.com/1298329396#comment,29,"[[5, '깨끗하고 7층 바다 보여서 좋은 밤 저렴하게 보냈어요 '], [5, '이...",바다 이정 도면 가격 대비 청결 상태도 이만 하면 프론트 친절 태풍 피해 하루 숙박...


In [59]:
hotel_keyword = pd.DataFrame()

hotel_keyword['place'] = hotel['카페명']
hotel_keyword['Top5'] = np.nan

for i in range(len(hotel)):
    noun = hotel['명사'][i].split(" ")
    count = Counter(noun).most_common(5)
    top5 = []
    for j in range(len(count)):
        top5.append(count[j][0])
    hotel_keyword['Top5'][i] = top5
    
for key in ['무료', '혜택', '친환경', '추천', '식재료', '제주산', '에너지', '환경', '비건']:
    hotel_keyword[key] = hotel['명사'].apply(lambda x: count_keyword(x, key))

/var/folders/_t/yrhf8bh167g1b6qb6gqjrf_c0000gn/T/ipykernel_5893/2736930490.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_keyword['Top5'][i] = top5


In [60]:
hotel_keyword

,place,Top5,무료,혜택,친환경,추천,식재료,제주산,에너지,환경,비건
0,라마다프라자 제주호텔,"[호텔, 객실, 화장실, 소리, 친절]",1,0,0,1,0,0,0,0,0
1,롯데호텔 제주,"[호텔, 친절, 직원, 서비스, 조식]",0,0,0,5,0,0,0,0,0
2,베니키아호텔 제주,"[호텔, 직원, 친절, 성비, 예약]",1,0,0,2,0,0,0,2,0
3,베니키아중문호텔,"[주차장, 주차, 성비, 관광지, 근처]",0,0,0,0,0,0,0,0,0
4,호텔더본 제주,"[호텔, 조식, 성비, 쿠폰, 가격]",6,0,0,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
165,은하호텔,"[온돌방, 호텔, 남자, 한라산, 등반]",0,0,0,0,0,0,0,0,0
166,라이프리버호텔,[],0,0,0,0,0,0,0,0,0
167,산방산호텔,"[호텔, 전체, 만족, 산방산, 벌레]",0,0,0,2,0,0,0,0,0
168,호텔케니모슬포,"[친절, 직원, 호텔, 바다, 가격]",0,0,0,2,0,0,0,2,0


# csv 파일로 내보내기

In [62]:
cafe_keyword.to_csv('./kakao_review/cafe_keyword.csv', header=True, index=False, encoding="UTF-8")
motel_keyword.to_csv('./kakao_review/motel_keyword.csv', header=True, index=False, encoding="UTF-8")
pension_keyword.to_csv('./kakao_review/pension_keyword.csv', header=True, index=False, encoding="UTF-8")
resort_keyword.to_csv('./kakao_review/resort_keyword.csv', header=True, index=False, encoding="UTF-8")
food_keyword.to_csv('./kakao_review/restaurant_keyword.csv', header=True, index=False, encoding="UTF-8")
tour_keyword.to_csv('./kakao_review/tour_keyword.csv', header=True, index=False, encoding="UTF-8")
hotel_keyword.to_csv('./kakao_review/hotel_keyword.csv', header=True, index=False, encoding="UTF-8")

In [84]:
cafe_keyword.to_csv('./kakao_review/cafe_keyword_plus.csv', header=True, index=False, encoding="UTF-8")
tour_keyword.to_csv('./kakao_review/tour_keyword_plus.csv', header=True, index=False, encoding="UTF-8")

In [91]:
food_keyword.to_csv('./kakao_review/food_keyword_plus.csv', header=True, index=False, encoding="UTF-8")

In [28]:
tour_keyword.to_csv('./kakao_review/tour_keyword_plus.csv', header=True, index=False, encoding="UTF-8")